# Understanding Output

Here we provide a highlevel overview of the bias and fairness metrics output by Aequitas. For detailed definitions of the metrics see [Understanding our metrics]()

In [1]:
import pandas as pd

def color_tf(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val == True:
        color = 'green'
    elif val == False:
        color = 'red'
    else:
        color = ''
    return 'color: %s' % color

In [2]:
df = pd.read_csv('../data/aequinas_test_20180223.csv')

### Input data

You input a csv using the command-line function `aequitas_audit`. The csv has columns for `entity_id`, `score`, and `label_value` as well as group attributes against which to test for disparities. In this cases we include `race`, `sex` and `age_cat`. This data comes from [this ProPublica analysis](https://github.com/propublica/compas-analysis) of the Northpoint COMPAS risk assessment algorithm. 

In [3]:
df[['model_id', 'entity_id', 'score', 'label_value','race', 'gender', 'age']].head()

,model_id,entity_id,score,label_value,race,gender,age
0,1,1,1,1,W,M,3
1,1,2,1,0,B,M,3
2,1,3,1,1,B,F,2
3,1,4,1,1,W,M,1
4,1,5,1,0,B,M,4


## Output
`aequitas_audit` returns multiple levels of analysis. Bias metrics, such as False Omission Rate, False Discovery Rate, False Positive Rate and False Negative Rate, look at individual groups.

---
### Bias measures


In [14]:
# load aequitas output
full_df = pd.read_csv('../data/compas_group_value_fairness.csv')

# filter to focus on race and bias metrics
df = full_df[['group_value', 'group_variable', 'FOmR', 'FDR', 'FPR', 'FNR']]\
                .query("group_variable == 'race'")

# stylize for nice presentation        
df.loc[df.index.max() + 1] = ["normalizer","normalizer", 1, 1,1,1]
df = df.style.set_properties(**{'border-style':'solid', 'border-color': 'white'})\
        .bar(subset=['FDR', 'FOmR', 'FPR', 'FNR'], align='mid', color='#4682B4')
    
display(df)

# As far as I can tell, there is not a built in way to have normalized bar lengths without 
# the inclusion of a row of 1s.
# e.g.
# df.data.drop(axis=0, index=11, inplace=True)
# pandas.io.formats.style.Styler also does not seem to appear on github. Will investigate al

,group_value,group_variable,FOmR,FDR,FPR,FNR
0,African-American,race,0.34954,0.370285,0.448468,0.279853
1,Asian,race,0.125,0.25,0.0869565,0.333333
2,Caucasian,race,0.288125,0.408665,0.234543,0.477226
3,Hispanic,race,0.288591,0.457895,0.214815,0.556034
4,Native American,race,0.166667,0.25,0.375,0.1
5,Other,race,0.302013,0.455696,0.147541,0.676692
6,normalizer,normalizer,1,1,1,1


### Disparity measures

Disparity are measured in relation to a base group (see [defining a base group]())

$$ Disparity =  \frac{metric_{group}}{metric_{base group}} $$

For details [link to more documentation]. 

In [26]:
full_df[['group_value', 'group_variable', 'FDR_disparity', 'FOmR_disparity',
       'FPR_disparity', 'FNR_disparity', 'Supervised Fairness']].query("group_variable == 'race'").style.applymap(color_tf)\
             .set_properties(**{'border-style':'solid', 'border-color': 'white'})\
# bars can't be normalized so are confusing.

,group_value,group_variable,FDR_disparity,FOmR_disparity,FPR_disparity,FNR_disparity,Supervised Fairness
0,African-American,race,0.906085,1.21315,1.91209,0.586416,False
1,Asian,race,0.611748,0.433839,0.370749,0.698482,False
2,Caucasian,race,1,1,1,1,True
3,Hispanic,race,1.12046,1.00162,0.915887,1.16514,True
4,Native American,race,0.611748,0.578453,1.59885,0.209544,False
5,Other,race,1.11508,1.0482,0.629057,1.41797,False


### Group-level fairness

By default, Aequitas says a metric is fair for a group if the group rate is within X% of the base group. 

Supervised fairness is comprised of Type I and Type II errors familar from statistics. In machine learning, these are captured by the False Discovery Rate or False Positive Rate (Type I) and False Ommision Rate or False Negative Rate (Type II). Supervised Fairness is True for a group, if each of these metrics is within the bounds of fairness compared to the base group.


In [19]:
full_df[['group_value', 'group_variable', 'FOmR Parity','FDR Parity',  'FPR Parity',
       'FNR Parity', 'Supervised Fairness']].query("group_variable == 'race'").style.applymap(color_tf)

,group_value,group_variable,FOmR Parity,FDR Parity,FPR Parity,FNR Parity,Supervised Fairness
0,African-American,race,True,True,False,False,False
1,Asian,race,False,False,False,False,False
2,Caucasian,race,True,True,True,True,True
3,Hispanic,race,True,True,True,True,True
4,Native American,race,False,False,False,False,False
5,Other,race,True,True,False,False,False


### Group-level unsupervised fairness

We can see the entire flow from group-level bias metrics to assertion of fairness using unsupervised fairness.

< more description > 

In [38]:
df_uf = full_df[['group_value', 'group_variable','PPR', 'PPrev','PPR_disparity', 'PPrev_disparity','Statistical Parity','Impact Parity','Unsupervised Fairness']]\
                .query("group_variable =='race'")
        
df_uf.loc[full_df.index.max() + 1] = ["normalizer","normalizer", 1, 1, "","","","",""]

df_uf.style.applymap(color_tf)\
     .set_properties(**{'border-style':'solid', 'border-color': 'white', 'white-space': 'nowrap'})\
     .bar(subset=['PPR', 'PPrev'], align='mid', color='#4682B4')
    

,group_value,group_variable,PPR,PPrev,PPR_disparity,PPrev_disparity,Statistical Parity,Impact Parity,Unsupervised Fairness
0,African-American,race,0.655412,0.588203,2.54567,1.69022,False,False,False
1,Asian,race,0.00241182,0.25,0.00936768,0.718384,False,False,False
2,Caucasian,race,0.257462,0.348003,1,1,True,True,True
3,Hispanic,race,0.0572807,0.298273,0.222482,0.857099,False,True,False
4,Native American,race,0.00361773,0.666667,0.0140515,1.91569,False,False,False
5,Other,race,0.0238167,0.209549,0.0925059,0.602147,False,False,False
11,normalizer,normalizer,1,1,,,,,


In [ ]:
# Code for sex ... probably not necessary
sex_df = full_df[['group_value', 'group_variable', 'FOmR', 'FDR', 'FPR', 'FNR']]\
        .query("group_variable == 'sex'")#, \
sex_df.loc[df.index.max() + 1] = ["normalizer","normalizer", 1, 1,1,1]
sex_df = sex_df.style.set_properties(**{'border-style':'solid', 'border-color': 'white'})\
        .bar(subset=['FDR', 'FOmR', 'FPR', 'FNR'], align='mid', color='#4682B4')
    
sex_df

In [ ]:
full_df[['group_value', 'group_variable', 'FOmR Parity','FDR Parity',  'FPR Parity',
       'FNR Parity', 'Supervised Fairness']].query("group_variable == 'sex'").style.applymap(color_tf)

## Group Variable Fairness

When there are many groups it is useful to have a broader overview of fairness. Aequitas summarizes parity and fairness for "group variables" (e.g. race, sex, age category). Similar to above, we report True if all parity measures of the subgroups are True. For example, below we see that Impact Parity is True for sex, this implies Impact Parity is fair for male and female (which can be verified above). 

In [26]:
group_var_df = pd.read_csv('../data/compas_group_variable_fairness.csv',index_col=0)
display(group_var_df.style.applymap(color_tf))

,model_id,parameter,group_variable,Impact Parity,FDR Parity,FPR Parity,FOmR Parity,FNR Parity,TypeI Parity,TypeII Parity,Unsupervised Fairness,Supervised Fairness
0,1,3317_abs,age_cat,False,True,False,False,False,False,False,False,False
1,1,3317_abs,race,False,False,False,False,False,False,False,False,False
2,1,3317_abs,sex,True,False,True,False,True,False,False,False,False
